In [1]:
import numpy as np
import matplotlib.pyplot as plt

from network_components import *
from network_logic import Model
from sample_data import *

import ipywidgets

In [6]:
def get_grid_predictions(x_min, x_max, nx, y_min, y_max, ny, model):
    x = np.linspace(x_min, x_max, nx)
    y = np.linspace(y_min, y_max, ny)
    xx, yy = np.meshgrid(x, y)
    points = np.vstack((
        xx.flatten(),
        yy.flatten()
    ))

    def softmax(Z):
        Z_exp = np.exp(Z)
        return Z_exp / np.sum(Z_exp, axis=0)
    preds = softmax(model(points))[0].reshape(ny, nx)
    
    return x, y, preds


x_min=-2.5
x_max=2.5
nx=50
y_min=-2.5
y_max=2.5
ny=50

def get_grid_predictions_wrapper(model):
    return get_grid_predictions(x_min, x_max, nx, y_min, y_max, ny, model)

In [10]:
def plot_grid_prediction(grid_prediction, X, Y):
    x, y, preds = grid_prediction
    
    cmap = 'RdYlBu'
    num_levels = 200

    plt.contourf(x, y, preds, levels=np.linspace(0, 1, num_levels), cmap=cmap)
    plt.colorbar(ticks=np.linspace(0, 1, 11)) 
    plt.contour(x, y, preds, levels=[.5], linewidths=.5)
    plt.scatter(X[0], X[1], c=Y, edgecolors='grey', cmap=cmap)
    plt.show()

In [8]:
# Get sample data
X, Y = circle()
# Transform Y into probabilities
Y_ = np.zeros((2, Y.shape[0]))
Y_[0, Y==1] = 1
Y_[1, Y==0] = 1

In [38]:
# gaussian: 0 hidden layers
# x_or: 1 hidden layer, 2 neurons
# circle: 1 hidden layer, 6 neurons

linear1 = LinearLayer(in_features=2, out_features=10)
relu1 = ReLU()
linear2 = LinearLayer(in_features=10, out_features=10)
relu2 = ReLU()
linear3 = LinearLayer(in_features=10, out_features=2)
smax_ce_loss = SoftmaxCrossEntropyLoss()

model = Model(
    linear1,
    relu1,
    linear2,
    relu2,
    linear3,
    loss_func=smax_ce_loss
)

In [39]:
num_epochs = 100
batch_size = 16
lr = .01

acc_list, loss_list, grid_prediction_list = model.fit(
    X, 
    Y_, 
    num_epochs, 
    batch_size, 
    lr,
    get_grid_predictions_wrapper
)

#Epoch 1/100: 100%|############| 100/100 [00:00<00:00, 110.67it/s, Acc = 0.0000]


In [40]:
def plot_grid_prediction_wrapper(x):
    plot_grid_prediction(grid_prediction_list[x], X, Y)

x = ipywidgets.IntSlider(
    value=0,
    min=0,
    max=len(grid_prediction_list)-1,
    description='Epoch:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

def p(x):
    print(f'Accuracy: {acc_list[x]:.3f}')
    
ipywidgets.interact(p, x=x)
ipywidgets.interact(plot_grid_prediction_wrapper, x=x)
print()

interactive(children=(IntSlider(value=0, continuous_update=False, description='Epoch:', max=99), Output()), _d…

interactive(children=(IntSlider(value=0, continuous_update=False, description='Epoch:', max=99), Output()), _d…